In [1]:
import json
import random
import pickle
import os

# function pool here
from templates_lib.filter import *
from templates_lib.func import *
from templates_lib.QA import QADataset, QATemplate, QAInstance, QAMCTemplate

### Static::Measurement::object_distance

In [2]:
myfilter = filter_all(
    filter_visiblity,
    filter_area,
    black_list_fn([
            "movable_object.trafficcone",
            "movable_object.barrier",
        ])
    )

myCap = Captioner("./captions.yaml")

roi_frame_only = True

single_obj_abs_dist = QATemplate(
    Q_temp="What is the distance between <obj> and the ego camera at frame <frame>? "
    "(return in meters, frame idx starts from 0).",
    A_temp="<abs_dist>",
    obj_mappers=[
        ("obj", myCap.obj_desc_fn(0)),
        ("frame", frame_idx_fn(0, roi_frame_only)),
        ("abs_dist", obj_cam_dist_fn(0)),
    ],
    obj_filter=myfilter,
    config={
        "num_objs": 1,
        "num_frames": 1,
        "QA_type": "single_obj_abs_dist"
        },
)

double_obj_abs_dist = QATemplate(
    Q_temp="What is the distance between <obj1> and <obj2> at frame <frame>? "
    "(return in meters, frame idx starts from 0).",
    A_temp="<abs_dist>",
    obj_mappers=[
        ("obj1", myCap.obj_desc_fn(0)),
        ("obj2", myCap.obj_desc_fn(1)),
        ("frame", frame_idx_fn(0, roi_frame_only)),
        ("abs_dist", obj_dist_between),
    ],
    obj_filter=myfilter,
    config={
        "num_objs": 2,
        "num_frames": 1,
        "QA_type": "double_obj_abs_dist"
    },
)

single_obj_minmax_dist = QATemplate(
    Q_temp="What is the <minmax> distance between <obj> "
    "and the ego camera between frame <frame_range>? "
    "(return in meters, frame idx starts from 0).",
    A_temp="<abs_dist>",
    obj_mappers=[
        ('minmax', minmax()),
        ("obj", myCap.obj_desc_fn(0)),
        ("frame_range", frame_range_fn(roi_frame_only)),
        ("abs_dist", obj_cam_dist_minmax),
    ],
    obj_filter=myfilter,
    config={
        "num_objs": 1,
        "num_frames": 5,
        "QA_type": "single_obj_minmax_dist"
        },
)

double_obj_minmax_dist = QATemplate(
    Q_temp="What is the <minmax> distance between <obj1> and <obj2> "
    "between frame <frame_range>? (return in meters, frame idx starts from 0).",
    A_temp="<abs_dist>",
    obj_mappers=[
        ('minmax', minmax()),
        ("obj1", myCap.obj_desc_fn(0)),
        ("obj2", myCap.obj_desc_fn(1)),
        ("frame_range", frame_range_fn(roi_frame_only)),
        ("abs_dist", obj_dist_between_minmax),
    ],
    obj_filter=myfilter,
    config={
        "num_objs": 2,
        "num_frames": 5,
        "QA_type": "double_obj_minmax_dist"
    },
)

multiple_obj_relative_dist = QAMCTemplate(
    Q_temp="Among the following objects, which one is the <minmax> the ego camera "
    f"at frame <frame> (frame idx starts from 0)? <{QAMCTemplate.OPT_PREFIX}>",
    A_temp=f"<{QAMCTemplate.ANS_PREFIX}>",
    obj_mappers=[
        ('minmax', minmax("closest to", "farthest from")),
        ("frame", frame_idx_fn(0, roi_frame_only)),
    ],
    obj_filter=myfilter,
    config={
        "num_objs": 2,
        "num_frames": 1,
        "opt_mapper_gen": myCap.obj_desc_fn,
        "ans_index_gen": index_of_minmax_dist,
        "QA_type": "multiple_obj_relative_dist"
    },
)

local_coords = QATemplate(
    Q_temp="What is the local coordinates of <obj> in the ego camera coordinate at frame <frame>? "
    "(return in meters, frame idx starts from 0, x points to the left, y points front, ignore vertical direction).",
    A_temp="<local_coords>",
    obj_mappers=[
        ("obj", myCap.obj_desc_fn(0)),
        ("frame", frame_idx_fn(0, roi_frame_only)),
        ("local_coords", obj_local_coords_fn(0)),
    ],
    obj_filter=myfilter,
    config={
        "num_objs": 1,
        "num_frames": 1,
        "QA_type": "local_coords"
        },
)

dummy = QATemplate(
    Q_temp="which dataset are you working on?",
    A_temp="nuScenes",
    obj_mappers=[],
    obj_filter=lambda x: True,
    config={
        "num_objs": 0,
        "num_frames": 0,
        "QA_type": "dummy"
    },
)

In [3]:
DS_ROOT = "./structured-data"
OUTPUT_DIR = "pairs"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_QWEN = f"{OUTPUT_DIR}/QA_pairs_qwen.json"
OUTPUT_JSON = f"{OUTPUT_DIR}/QA_pairs.json"
TEST_SPLIT = 0.2
ds = QADataset(DS_ROOT, myCap)

random.seed()
qas: list[QAInstance] = []
num_qas = 10_000
qa_temps = [
    single_obj_abs_dist,
    double_obj_abs_dist,
    single_obj_minmax_dist,
    double_obj_minmax_dist,
    multiple_obj_relative_dist,
    local_coords,
    # dummy,
]


sc = random.choice(ds.scenes[1:2])
print(f"scene: {sc.name}")

stats = {}

while len(qas) < num_qas:
    temp = random.choice(qa_temps)
    try:
        qa = temp(sc)
    except Exception as e:
        # print(f"Error: {e}")
        continue
    qas.append(qa)
    stats[qa.QA_type] = stats.get(qa.QA_type, 0) + 1
    if len(qas) % 500 == 0:
        print(f"Generated {len(qas)} QAs, stats: {stats}")

num_test = int(len(qas) * TEST_SPLIT)
qas_train = qas[:-num_test]
qas_test = qas[-num_test:]
all_frames = not roi_frame_only

with open(OUTPUT_QWEN.replace(".", ".test."), "w") as f:
    json.dump(
        [qa.qwen_format(all_frames=all_frames) for qa in qas_test], f, indent=2
    )
with open(OUTPUT_QWEN.replace(".", ".train."), "w") as f:
    json.dump(
        [qa.qwen_format(all_frames=all_frames) for qa in qas_train], f, indent=2
    )
with open(OUTPUT_JSON.replace(".", ".test."), "w") as f:
    qas_dumps = [qa.dump() for qa in qas_test]
    json.dump(qas_dumps, f, indent=2)
with open(OUTPUT_JSON.replace(".", ".train."), "w") as f:
    qas_dumps = [qa.dump() for qa in qas_train]
    json.dump(qas_dumps, f, indent=2)


Found 850 scenes in ./structured-data
scene: 9c83e438973e4824853e6a38928ca4ff
Generated 500 QAs, stats: {'local_coords': 120, 'double_obj_minmax_dist': 61, 'single_obj_abs_dist': 105, 'single_obj_minmax_dist': 196, 'multiple_obj_relative_dist': 5, 'double_obj_abs_dist': 13}
Generated 1000 QAs, stats: {'local_coords': 237, 'double_obj_minmax_dist': 119, 'single_obj_abs_dist': 209, 'single_obj_minmax_dist': 397, 'multiple_obj_relative_dist': 14, 'double_obj_abs_dist': 24}
Generated 1500 QAs, stats: {'local_coords': 354, 'double_obj_minmax_dist': 175, 'single_obj_abs_dist': 328, 'single_obj_minmax_dist': 585, 'multiple_obj_relative_dist': 23, 'double_obj_abs_dist': 35}
Generated 2000 QAs, stats: {'local_coords': 457, 'double_obj_minmax_dist': 236, 'single_obj_abs_dist': 432, 'single_obj_minmax_dist': 793, 'multiple_obj_relative_dist': 35, 'double_obj_abs_dist': 47}
Generated 2500 QAs, stats: {'local_coords': 571, 'double_obj_minmax_dist': 295, 'single_obj_abs_dist': 530, 'single_obj_minma